
# DSA Guide Bot (CrewAI) — Final Fixed (No `context` arg)

This Colab-ready notebook builds a DSA mentor bot with CrewAI + Gradio.

**Fixes in this version**
- Avoids CrewAI `config` parsing error by **not** using `context=`.  
  We embed the shared context directly in each task's `description`.
- Version pins for compatibility.
- Safer default models (`gpt-4o-mini` or `llama-3.1-8b-instant`).
- Gradio shows full tracebacks.


In [ ]:

# %%capture
!pip -q install "crewai==0.51.1" "langchain-openai>=0.1.14" "langchain-groq>=0.1.3" gradio pydantic==2.*


In [ ]:

import os

# Paste ONE key below (or set in Colab's environment)
# os.environ['OPENAI_API_KEY'] = 'sk-...'
# os.environ['GROQ_API_KEY']   = 'gsk_...'

from typing import Optional

USE_PROVIDER: Optional[str] = None  # 'openai' or 'groq' or None to autodetect

openai_key = os.getenv('OPENAI_API_KEY', '').strip()
groq_key = os.getenv('GROQ_API_KEY', '').strip()

if USE_PROVIDER is None:
    if groq_key:
        USE_PROVIDER = 'groq'
    elif openai_key:
        USE_PROVIDER = 'openai'
    else:
        raise RuntimeError("No API key found. Set OPENAI_API_KEY or GROQ_API_KEY.")

print("Using provider:", USE_PROVIDER.upper())


In [ ]:

from crewai import Agent, Task, Crew, Process

# LLM selection
if USE_PROVIDER == 'groq':
    from langchain_groq import ChatGroq
    llm = ChatGroq(model="llama-3.1-8b-instant")
elif USE_PROVIDER == 'openai':
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini")
else:
    raise ValueError("Invalid provider.")

# Agents
mentor = Agent(
    role="DSA Mentor",
    goal=("Analyze DSA problems, classify them, and propose the optimal approach with key insights."),
    backstory=("Expert DSA tutor who explains clearly with patterns and edge cases."),
    llm=llm, allow_delegation=False, verbose=True,
)
coder = Agent(
    role="Code Generator",
    goal=("Write clean, optimal, self-contained code in Python, C++, or Java."),
    backstory="Competitive programmer and SE.",
    llm=llm, allow_delegation=False, verbose=True,
)
reviewer = Agent(
    role="Complexity Reviewer",
    goal="Provide correct Big-O time/space with trade-offs.",
    backstory="Fast, accurate complexity checks.",
    llm=llm, allow_delegation=False, verbose=True,
)
tester = Agent(
    role="Test Designer",
    goal=("Design concise tests incl. edge cases; if Python, provide a mini harness."),
    backstory="Thinks like an online judge.",
    llm=llm, allow_delegation=False, verbose=True,
)


In [ ]:

from textwrap import dedent

def solve_with_crew(problem: str, language: str = "Python") -> str:
    language = (language or "Python").strip().title()
    if language not in {"Python", "C++", "Java"}:
        language = "Python"

    common = dedent(f"""
    Problem:
    {problem}

    Target language: {language}

    Output sections (use these headings):
    1) Problem Understanding & Pattern
    2) Step-by-step Approach
    3) Final Code ({language})
    4) Complexity Analysis
    5) Tests
    """)

    t1 = Task(
        description = common + "\nAnalyze category/pattern, outline an optimal algorithm, key observations and edge cases.",
        agent=mentor,
        expected_output="Short analysis with category + approach outline.",
    )
    t2 = Task(
        description = common + "\nWrite clean, optimal, self-contained code in the chosen language. Avoid interactive input.",
        agent=coder,
        expected_output="Single code block solving the problem.",
    )
    t3 = Task(
        description = common + "\nProvide Big-O time and space complexity plus brief trade-offs.",
        agent=reviewer,
        expected_output="Time complexity and space complexity.",
    )
    t4 = Task(
        description = common + "\nDesign tests incl. edge cases. If Python, add a small test harness.",
        agent=tester,
        expected_output="Bullet list of tests (+ Python snippet if applicable).",
    )

    crew = Crew(
        agents=[mentor, coder, reviewer, tester],
        tasks=[t1, t2, t3, t4],
        process=Process.sequential,
        verbose=True,
    )
    result = crew.kickoff()
    return str(result)


In [ ]:

import gradio as gr, traceback

def ui_run(problem, language):
    try:
        if not problem or not problem.strip():
            return "Please enter a problem statement."
        return solve_with_crew(problem, language)
    except Exception:
        return "### Error\n````\n" + "".join(traceback.format_exc()) + "\n````"

with gr.Blocks(title="DSA Guide Bot (CrewAI)") as demo:
    gr.Markdown("## DSA Guide Bot — CrewAI\nEnter your DSA problem and choose a language.")
    lang = gr.Dropdown(choices=["Python", "C++", "Java"], value="Python", label="Language")
    prob = gr.Textbox(label="Problem Statement", lines=8, placeholder="Describe your DSA problem here...")
    run = gr.Button("Solve")
    out = gr.Markdown()
    run.click(ui_run, inputs=[prob, lang], outputs=out)

demo.queue().launch()
